In [128]:
import csv
from pathlib import Path
import pandas as pd
from pandas import DataFrame
import os
from dotenv import load_dotenv
from datetime import datetime
from datetime import date
from datetime import timedelta
import requests
import json

import matplotlib.pyplot as plt
import hvplot.pandas
import plotly.express as px
import panel as pn
from panel.interact import interact
from panel import widgets
import numpy as np

%matplotlib inline

# Initialize the Panel Extensions (for Plotly)
pn.extension('plotly')

# Input Portfolio
Read in the input portfolio CSV file and output a portfolio dataframe
* Inputs:
    * Portfolio CSV file
    * Format: Ticker, Transaction (B=buy, S=Sell), Date, Transaction Price, Number of Shares
    * Assumptions: Date unsorted, no '$' sign, no commas
* Outputs:
    * Portfolio database
    * Format: Ticker, Transaction (B=buy, S=Sell), Date, Transaction Price, Number of Shares, Transaction Cost
    * Database sorted by ascending date

In [2]:
# Set the path for the CSV file
input_file_path = Path("TestPortfolio.csv")

In [3]:
#Create empty list to store row data from CSV
portfolio_transactions = []

In [4]:
#Open the CSV file; read in row data; calculate transaction cost per transacation
with open(input_file_path, 'r') as csvfile:
    #Set reader object
    csvreader = csv.reader(csvfile, delimiter = ',')
 
    #Read the header
    csv_header = next(csvreader)
    
    #Create new column
    csv_header.append("Transaction Cost")
    portfolio_transactions.append(csv_header)

    #Iterate through the data 
    for row in csvreader:
        #Calculate transaction cost
        cost = float(row[3]) * float(row[4])
        
        #Append transaction cost to list
        row.append(cost)
        portfolio_transactions.append(row)

In [5]:
#Convert transaction list to dataframe
portfolio_df = pd.DataFrame(portfolio_transactions, columns=['Ticker', 'Transaction', 'Date', 'Transaction Price', 'Number of shares', 'Transaction Cost'])
#Drop first row (duplicate header)
portfolio_df=portfolio_df.drop(portfolio_df.index[0])
portfolio_df

,Ticker,Transaction,Date,Transaction Price,Number of shares,Transaction Cost
1,AAPL,B,3/5/12,77.8,100,7780
2,AAPL,B,4/1/13,60.57,100,6057
3,TSLA,B,9/23/13,189.9,100,18990
4,TSLA,S,11/18/13,122.38,75,9178.5
5,TSLA,B,5/9/16,208,30,6240
6,GOOG,B,8/7/17,915.39,10,9153.9
7,AAPL,S,3/16/20,240,50,12000


In [6]:
#Convert Date to datetime and set as index
portfolio_df['Date']=pd.to_datetime(portfolio_df['Date'])
portfolio_df.set_index(['Date'], inplace=True)
portfolio_df

,Ticker,Transaction,Transaction Price,Number of shares,Transaction Cost
Date,,,,,
2012-03-05,AAPL,B,77.8,100,7780
2013-04-01,AAPL,B,60.57,100,6057
2013-09-23,TSLA,B,189.9,100,18990
2013-11-18,TSLA,S,122.38,75,9178.5
2016-05-09,TSLA,B,208,30,6240
2017-08-07,GOOG,B,915.39,10,9153.9
2020-03-16,AAPL,S,240,50,12000


In [7]:
# Sort dataframe in ascending order
portfolio_df.sort_index(inplace=True)
portfolio_df

,Ticker,Transaction,Transaction Price,Number of shares,Transaction Cost
Date,,,,,
2012-03-05,AAPL,B,77.8,100,7780
2013-04-01,AAPL,B,60.57,100,6057
2013-09-23,TSLA,B,189.9,100,18990
2013-11-18,TSLA,S,122.38,75,9178.5
2016-05-09,TSLA,B,208,30,6240
2017-08-07,GOOG,B,915.39,10,9153.9
2020-03-16,AAPL,S,240,50,12000


# Pull Stock Data
Using the portfolio database, determine the number of unique stocks and pull the stock history.
* Inputs:
    * Portfolio database (portfolio_df)
* Outputs:
    * Ticker Stat database (ticker_stats_df) (Tickers as Rows)
    * Transposed Ticker Stat Database (ticker_stats_df_T) (Tickers as Columns)
    * Dailiy Closing Price database (closing_df)

In [8]:
# Set environment variables from the .env file
# rel_path = Path('../../Python') / '.env'
# load_dotenv(rel_path)
load_dotenv("/Users/heenaroy/Desktop/.env")

True

In [9]:
# Grab API Key for https://financialmodelingprep.com
stock_key = os.getenv("FINANCIAL_MODEL_KEY")

In [10]:
# Create empty list for stock tickers
stock_list=[]

In [11]:
# Determine unique stocks
group=portfolio_df.groupby('Ticker')
# Store unique stocks to stock list
group.groups.keys()
stock_list=list(group.groups)
stock_list

['AAPL', 'GOOG', 'TSLA']

In [12]:
#Create new dataframe resetting date index
portfolio2_df=portfolio_df.reset_index()
portfolio2_df

,Date,Ticker,Transaction,Transaction Price,Number of shares,Transaction Cost
0,2012-03-05,AAPL,B,77.8,100,7780
1,2013-04-01,AAPL,B,60.57,100,6057
2,2013-09-23,TSLA,B,189.9,100,18990
3,2013-11-18,TSLA,S,122.38,75,9178.5
4,2016-05-09,TSLA,B,208,30,6240
5,2017-08-07,GOOG,B,915.39,10,9153.9
6,2020-03-16,AAPL,S,240,50,12000


In [13]:
#Find the min dates per tikcer
min_stock_dates=portfolio2_df.groupby('Ticker').min()
min_stock_dates

,Date,Transaction,Transaction Price,Number of shares,Transaction Cost
Ticker,,,,,
AAPL,2012-03-05,B,240,100,6057.0
GOOG,2017-08-07,B,915.39,10,9153.9
TSLA,2013-09-23,B,122.38,100,6240.0


In [14]:
#Store yesterday's date
yesterday = date.today() - timedelta(days=1)
yesterday = yesterday.strftime("%Y-%m-%d")
yesterday

'2020-06-16'

In [15]:
#Define Main Loop Variables
stock_dict={}            # Create dictionary to store stock data
count=0                  # Create counter as index for looping through stock list

#For each ticker in the stock list, pull the stock data
for ticker in stock_list:
    #Define new dictionary key set per ticker
    stock_dict[ticker]={
        'PE Ratio'            : 0,
        'Profit Margin'       : 0,
        'Cash'                : 0,
        'Beta'                : 0,
        'Last Dividend'       : 0,
        'Industry'            : 0,
        'Sector'              : 0,
        'Daily Closing Price' : 0,
        }
    
######
    #Get Beta, Last Div, Industry and Sector
    url = f"https://financialmodelingprep.com/api/v3/profile/{ticker}?apikey={stock_key}"
    requests.get(url).content
    parsed = json.loads(requests.get(url).content)
    
    #Store stock data in dictionary
    stock_dict[ticker]['Beta']=parsed[0]['beta']
    stock_dict[ticker]['Last Dividend']=parsed[0]['lastDiv']
    stock_dict[ticker]['Industry']=parsed[0]['industry']
    stock_dict[ticker]['Sector']=parsed[0]['sector']
    

######
    # Grab PE Ratio
    url = f"https://financialmodelingprep.com/api/v3/key-metrics/{ticker}?apikey={stock_key}"
    requests.get(url).content
    parsed = json.loads(requests.get(url).content)
    
    # Store PE Ratio in dictionary
    stock_dict[ticker]['PE Ratio']=parsed[0]['peRatio']
    
    
######    
    # Grab Profit Margin
    url = f"https://financialmodelingprep.com/api/v3/ratios/{ticker}?apikey={stock_key}"
    requests.get(url).content
    parsed = json.loads(requests.get(url).content)
    
    # Store Profit Margin in dictionary
    stock_dict[ticker]['Profit Margin']=parsed[0]['netProfitMargin']
    
   
###### 
    # Grab Cash
    url = f"https://financialmodelingprep.com/api/v3/balance-sheet-statement/{ticker}?apikey={stock_key}"
    requests.get(url).content
    parsed = json.loads(requests.get(url).content)
    
    # Store Profit Margin in dictionary
    stock_dict[ticker]['Cash']=parsed[0]['cashAndCashEquivalents']

     
######
    #Grab the first purchase date for current ticker
    datetime_obj=datetime.strptime(str(min_stock_dates['Date'][ticker]),"%Y-%m-%d %H:%M:%S")
    date_obj=datetime_obj.date()
    first_purchase_date=str(date_obj)
    
    # Grab the Daily Closing Prices between the first purchase date and yesterday
    url = f"https://financialmodelingprep.com/api/v3/historical-price-full/{ticker}?from={first_purchase_date}&to={yesterday}&apikey={stock_key}"
    requests.get(url).content
    parsed = json.loads(requests.get(url).content)
 
    ### Daily Stock Pull Variables
    count2=0                 # Create counter as index for looping through dictionary of stock data    
    daily_dict={}            # Create empty dictionary to store daily stock data before appending to daily_list_of_dict
    daily_list_of_dict = []  # Create empty list to store dictionary info from daily_dict before storing to stock_dict

    # Loop through historical data and grab daily closing date and price
    for i in parsed['historical']:
        daily_dict['Date'] = parsed['historical'][count2]['date']
        daily_dict['Price'] = parsed['historical'][count2]['close']
        daily_list_of_dict.append(daily_dict.copy())
        count2+=1

    #Store historical stock data to stock_dict dictionary
    stock_dict[ticker]['Daily Closing Price']=daily_list_of_dict
    
    
    #increment counter
    count+=1 

In [16]:
#Copy stock_dict so that main loop above doesn't have to be run multiple times
stock_dict_copy = stock_dict
stock_dict_copy

{'AAPL': {'PE Ratio': 20.81351450883162,
  'Profit Margin': 0.21238094505984456,
  'Cash': 48844000000,
  'Beta': 1.228499,
  'Last Dividend': 3.08,
  'Industry': 'Consumer Electronics',
  'Sector': 'Technology',
  'Daily Closing Price': [{'Date': '2020-06-16', 'Price': 352.08},
   {'Date': '2020-06-15', 'Price': 342.99},
   {'Date': '2020-06-12', 'Price': 338.8},
   {'Date': '2020-06-11', 'Price': 335.9},
   {'Date': '2020-06-10', 'Price': 352.84},
   {'Date': '2020-06-09', 'Price': 343.99},
   {'Date': '2020-06-08', 'Price': 333.46},
   {'Date': '2020-06-05', 'Price': 331.5},
   {'Date': '2020-06-04', 'Price': 322.32},
   {'Date': '2020-06-03', 'Price': 325.12},
   {'Date': '2020-06-02', 'Price': 323.34},
   {'Date': '2020-06-01', 'Price': 321.85},
   {'Date': '2020-05-29', 'Price': 317.94},
   {'Date': '2020-05-28', 'Price': 318.25},
   {'Date': '2020-05-27', 'Price': 318.11},
   {'Date': '2020-05-26', 'Price': 316.73},
   {'Date': '2020-05-22', 'Price': 318.89},
   {'Date': '2020-0

In [17]:
# Create DataFrame showing Beta, P/E Ratio, Profit Margin, 
# Cash, and current Dividends per ticker
ticker_stats_df=DataFrame(stock_dict_copy)
ticker_stats_df.drop('Daily Closing Price', inplace=True)
ticker_stats_df

,AAPL,GOOG,TSLA
Beta,1.2285,1.02276,0.580101
Cash,48844000000,18498000000,6268000000
Industry,Consumer Electronics,Internet Content & Information,Auto Manufacturers
Last Dividend,3.08,0,0
PE Ratio,20.8135,14.6956,-131.582
Profit Margin,0.212381,0.212181,-0.035072
Sector,Technology,Communication Services,Consumer Cyclical


In [56]:
# Create a transpose of ticker_stats_df
ticker_stats_df_T=ticker_stats_df.T
ticker_stats_df_T=ticker_stats_df_T[['PE Ratio', 'Profit Margin', 'Cash', 'Beta', 'Last Dividend', 'Industry', 'Sector']]
ticker_stats_df_T

,PE Ratio,Profit Margin,Cash,Beta,Last Dividend,Industry,Sector
AAPL,20.8135,0.212381,48844000000,1.2285,3.08,Consumer Electronics,Technology
GOOG,14.6956,0.212181,18498000000,1.02276,0,Internet Content & Information,Communication Services
TSLA,-131.582,-0.035072,6268000000,0.580101,0,Auto Manufacturers,Consumer Cyclical


In [57]:
# Closing Data

In [58]:
# DELETE??

# Create DataFrame showing Daily Closing Price per ticker
daily_closing_df=DataFrame(stock_dict_copy)
daily_closing_df.drop(['Beta', 'Cash', 'Industry', 'Last Dividend', 'PE Ratio', 'Profit Margin', 'Sector'], inplace=True)
daily_closing_df

,AAPL,GOOG,TSLA
Daily Closing Price,"[{'Date': '2020-06-16', 'Price': 352.08}, {'Da...","[{'Date': '2020-06-16', 'Price': 1442.72}, {'D...","[{'Date': '2020-06-16', 'Price': 982.13}, {'Da..."


In [59]:
# DELETE??

aapl_df = pd.DataFrame(daily_closing_df['AAPL'][0])
#aapl_df.rename(columns={'Date' : 'AAPL Closing Date', 'Price' : 'AAPL Closing Price'}, inplace=True)
aapl_df.rename(columns={'Price' : 'AAPL Closing Price'}, inplace=True)
aapl_df.head()

,Date,AAPL Closing Price
0,2020-06-16,352.08
1,2020-06-15,342.99
2,2020-06-12,338.80
3,2020-06-11,335.90
4,2020-06-10,352.84


In [60]:
# DELETE??

goog_df = pd.DataFrame(daily_closing_df['GOOG'][0])
#goog_df.rename(columns={'Date' : 'GOOG Closing Date', 'Price' : 'GOOG Closing Price'}, inplace=True)
goog_df.rename(columns={'Price' : 'GOOG Closing Price'}, inplace=True)
goog_df.head()

,Date,GOOG Closing Price
0,2020-06-16,1442.72
1,2020-06-15,1419.85
2,2020-06-12,1413.18
3,2020-06-11,1403.84
4,2020-06-10,1465.85


In [61]:
# DELETE??

tsla_df = pd.DataFrame(daily_closing_df['TSLA'][0])
#tsla_df.rename(columns={'Date' : 'TSLA Closing Date', 'Price' : 'TSLA Closing Price'}, inplace=True)
tsla_df.rename(columns={'Price' : 'TSLA Closing Price'}, inplace=True)
tsla_df.head()

,Date,TSLA Closing Price
0,2020-06-16,982.13
1,2020-06-15,990.90
2,2020-06-12,935.28
3,2020-06-11,972.84
4,2020-06-10,1025.05


In [62]:
# DELETE??

closing_df = aapl_df.set_index('Date').join(goog_df.set_index('Date')).join(tsla_df.set_index('Date'))
closing_df.head()

,AAPL Closing Price,GOOG Closing Price,TSLA Closing Price
Date,,,
2020-06-16,352.08,1442.72,982.13
2020-06-15,342.99,1419.85,990.90
2020-06-12,338.80,1413.18,935.28
2020-06-11,335.90,1403.84,972.84
2020-06-10,352.84,1465.85,1025.05


# Insert Alex's Code

In [63]:
# Convert daily closing price data (stock_dict_copy['Daily Closing Price'])
# into a new dataframe (closing_price_df) with only Date, Ticker, and Close columns
date=[]
price=[]
stock_name=[]

for stock in stock_dict_copy:
    for d in stock_dict_copy[stock]['Daily Closing Price']:
        stock_name.append(stock)
        date.append(datetime.strptime(d['Date'],'%Y-%m-%d'))
        price.append(d['Price'])
        
closing_price_df= pd.DataFrame(list(zip(date, stock_name, price)), columns=['Date', 'Ticker', 'Close'])
closing_price_df

,Date,Ticker,Close
0,2020-06-16,AAPL,352.08
1,2020-06-15,AAPL,342.99
2,2020-06-12,AAPL,338.80
3,2020-06-11,AAPL,335.90
4,2020-06-10,AAPL,352.84
5,2020-06-09,AAPL,343.99
6,2020-06-08,AAPL,333.46
7,2020-06-05,AAPL,331.50
8,2020-06-04,AAPL,322.32
9,2020-06-03,AAPL,325.12


In [98]:
closing_price_df=closing_price_df.sort_values('Date')
closing_price_df

,Date,Ticker,Close
2084,2012-03-05,AAPL,76.17
2083,2012-03-06,AAPL,75.75
2082,2012-03-07,AAPL,75.81
2081,2012-03-08,AAPL,77.43
2080,2012-03-09,AAPL,77.88
2079,2012-03-12,AAPL,78.86
2078,2012-03-13,AAPL,81.16
2077,2012-03-14,AAPL,84.23
2076,2012-03-15,AAPL,83.65
2075,2012-03-16,AAPL,83.65


In [99]:
#Merge the input portfolio dataframe to the closing price dataframe
portfolio3_df = pd.merge(portfolio_df, closing_price_df, how='right', left_on=['Date', 'Ticker'], right_on=['Date', 'Ticker'])
portfolio3_df

,Date,Ticker,Transaction,Transaction Price,Number of shares,Transaction Cost,Close
0,2012-03-05,AAPL,B,77.8,100,7780,76.17
1,2013-04-01,AAPL,B,60.57,100,6057,61.27
2,2013-09-23,TSLA,B,189.9,100,18990,181.11
3,2013-11-18,TSLA,S,122.38,75,9178.5,121.58
4,2016-05-09,TSLA,B,208,30,6240,208.92
5,2017-08-07,GOOG,B,915.39,10,9153.9,929.36
6,2020-03-16,AAPL,S,240,50,12000,242.21
7,2012-03-06,AAPL,NaN,NaN,NaN,NaN,75.75
8,2012-03-07,AAPL,NaN,NaN,NaN,NaN,75.81
9,2012-03-08,AAPL,NaN,NaN,NaN,NaN,77.43


In [100]:
portfolio3_df=portfolio3_df.sort_values('Date')
portfolio3_df=portfolio3_df.reset_index(drop=True)
portfolio3_df

,Date,Ticker,Transaction,Transaction Price,Number of shares,Transaction Cost,Close
0,2012-03-05,AAPL,B,77.8,100,7780,76.17
1,2012-03-06,AAPL,NaN,NaN,NaN,NaN,75.75
2,2012-03-07,AAPL,NaN,NaN,NaN,NaN,75.81
3,2012-03-08,AAPL,NaN,NaN,NaN,NaN,77.43
4,2012-03-09,AAPL,NaN,NaN,NaN,NaN,77.88
5,2012-03-12,AAPL,NaN,NaN,NaN,NaN,78.86
6,2012-03-13,AAPL,NaN,NaN,NaN,NaN,81.16
7,2012-03-14,AAPL,NaN,NaN,NaN,NaN,84.23
8,2012-03-15,AAPL,NaN,NaN,NaN,NaN,83.65
9,2012-03-16,AAPL,NaN,NaN,NaN,NaN,83.65


In [101]:
portfolio3_df['Number of shares']=portfolio3_df['Number of shares'].fillna(0)
portfolio3_df

,Date,Ticker,Transaction,Transaction Price,Number of shares,Transaction Cost,Close
0,2012-03-05,AAPL,B,77.8,100,7780,76.17
1,2012-03-06,AAPL,NaN,NaN,0,NaN,75.75
2,2012-03-07,AAPL,NaN,NaN,0,NaN,75.81
3,2012-03-08,AAPL,NaN,NaN,0,NaN,77.43
4,2012-03-09,AAPL,NaN,NaN,0,NaN,77.88
5,2012-03-12,AAPL,NaN,NaN,0,NaN,78.86
6,2012-03-13,AAPL,NaN,NaN,0,NaN,81.16
7,2012-03-14,AAPL,NaN,NaN,0,NaN,84.23
8,2012-03-15,AAPL,NaN,NaN,0,NaN,83.65
9,2012-03-16,AAPL,NaN,NaN,0,NaN,83.65


In [102]:
# Convert Number of shares and Transaction Price to float
portfolio3_df['Number of shares']=portfolio3_df['Number of shares'].astype(float)
portfolio3_df['Transaction Price']=portfolio3_df['Transaction Price'].astype(float)

In [103]:
# Calculate total number of shares
portfolio3_df['Net Shares'] = portfolio3_df.apply(lambda row: row['Number of shares'] if row.Transaction == "B" else row['Number of shares'] * -1, axis=1)
portfolio3_df['Cumulative Shares'] = portfolio3_df[['Net Shares','Ticker']].groupby('Ticker').cumsum()
# portfolio3_df.drop('Net Shares', inplace=True)
portfolio3_df

,Date,Ticker,Transaction,Transaction Price,Number of shares,Transaction Cost,Close,Net Shares,Cumulative Shares
0,2012-03-05,AAPL,B,77.8,100.0,7780,76.17,100.0,100.0
1,2012-03-06,AAPL,NaN,NaN,0.0,NaN,75.75,-0.0,100.0
2,2012-03-07,AAPL,NaN,NaN,0.0,NaN,75.81,-0.0,100.0
3,2012-03-08,AAPL,NaN,NaN,0.0,NaN,77.43,-0.0,100.0
4,2012-03-09,AAPL,NaN,NaN,0.0,NaN,77.88,-0.0,100.0
5,2012-03-12,AAPL,NaN,NaN,0.0,NaN,78.86,-0.0,100.0
6,2012-03-13,AAPL,NaN,NaN,0.0,NaN,81.16,-0.0,100.0
7,2012-03-14,AAPL,NaN,NaN,0.0,NaN,84.23,-0.0,100.0
8,2012-03-15,AAPL,NaN,NaN,0.0,NaN,83.65,-0.0,100.0
9,2012-03-16,AAPL,NaN,NaN,0.0,NaN,83.65,-0.0,100.0


# Testing code for Avg Cost per Share

In [32]:
# x=portfolio3_df.shift()['Transaction Cost']
# y=portfolio3_df['Transaction Cost']
# y

In [33]:
# portfolio3_df.groupby('Ticker')['Transaction Cost'].apply(lambda x: x.shift(1) - x)

In [34]:
# portfolio3_df.groupby('Ticker')['Avg Price per Share'] = portfolio3_df.apply(lambda row: ((row.shift()['Transaction Cost'] + row['Transaction Cost'])/row['Cumulative Shares']) if row.Transaction == "B" else row.shift()['Avg Price per Share'], axis=1)
# portfolio3_df

In [ ]:
# (aapl_prev_trans_cost + portfolio3_df['Transaction Cost'][i])/(portfolio3_df['Cumulative Shares'][i])

In [ ]:
# aapl_first_time=0
# count=0
# avg_price=[]
# aapl_avg_price=0
# tran=len(portfolio3_df)
# for i in range(tran):
#     if portfolio3_df['Ticker'][i] == "AAPL":
#         if aapl_first_time == 0:
#             aapl_avg_price = portfolio3_df['Transaction Price'][i]
#             aapl_prev_trans_cost = portfolio3_df['Transaction Cost'][i]
#             aapl_first_time=1
#         else:
#             if portfolio3_df['Transaction'][i] == "B":               
#                 aapl_avg_price = (aapl_prev_trans_cost + portfolio3_df['Transaction Cost'][i])/(portfolio3_df['Cumulative Shares'][i])
#                 aapl_prev_trans_cost = aapl_prev_trans_cost + portfolio3_df['Transaction Cost'][i]  
#             elif portfolio3_df['Transaction'][i] == "S":
#                 aapl_avg_price = aapl_avg_price
#             else:
#                 aapl_avg_price = aapl_avg_price
        
#         avg_price.append(aapl_avg_price)
#     else:
#         avg_price.append(0)

#     count+=1
    
# portfolio3_df['Avg Price per Share']=pd.DataFrame(avg_price)
# portfolio3_df

# Code for Avg Cost per Share

In [104]:
# Code for Avg Cost per Share (By Individual Ticker DF)

def avg_price(df):
    first_time=0
    avg_price_list=[]
    tran=len(df)

    for i in range(tran):
        if first_time == 0:
            avg_price = df['Transaction Price'][i]
            prev_trans_cost = df['Transaction Cost'][i]
            first_time=1
        else:
            if df['Transaction'][i] == "B":
                avg_price = (prev_trans_cost + df['Transaction Cost'][i])/(df['Cumulative Shares'][i])
                prev_trans_cost = prev_trans_cost + df['Transaction Cost'][i]
            elif df['Transaction'][i] == "S":
                avg_price = avg_price
            else:
                avg_price = avg_price

        avg_price_list.append(avg_price)

    return avg_price_list

In [105]:
# Create DF for AAPL

aapl3_df = portfolio3_df[portfolio3_df['Ticker']=='AAPL']
aapl3_df=aapl3_df.reset_index(drop=True)

# Run function to create Avg Cost per Share for AAPL

aapl3_df['Avg Price per Share']=pd.DataFrame(avg_price(aapl3_df))

aapl3_df

,Date,Ticker,Transaction,Transaction Price,Number of shares,Transaction Cost,Close,Net Shares,Cumulative Shares,Avg Price per Share
0,2012-03-05,AAPL,B,77.8,100.0,7780,76.17,100.0,100.0,77.800
1,2012-03-06,AAPL,NaN,NaN,0.0,NaN,75.75,-0.0,100.0,77.800
2,2012-03-07,AAPL,NaN,NaN,0.0,NaN,75.81,-0.0,100.0,77.800
3,2012-03-08,AAPL,NaN,NaN,0.0,NaN,77.43,-0.0,100.0,77.800
4,2012-03-09,AAPL,NaN,NaN,0.0,NaN,77.88,-0.0,100.0,77.800
5,2012-03-12,AAPL,NaN,NaN,0.0,NaN,78.86,-0.0,100.0,77.800
6,2012-03-13,AAPL,NaN,NaN,0.0,NaN,81.16,-0.0,100.0,77.800
7,2012-03-14,AAPL,NaN,NaN,0.0,NaN,84.23,-0.0,100.0,77.800
8,2012-03-15,AAPL,NaN,NaN,0.0,NaN,83.65,-0.0,100.0,77.800
9,2012-03-16,AAPL,NaN,NaN,0.0,NaN,83.65,-0.0,100.0,77.800


In [106]:
# Create DF for TSLA

tsla3_df = portfolio3_df[portfolio3_df['Ticker']=='TSLA']
tsla3_df=tsla3_df.reset_index(drop=True)

# Run function to create Avg Cost per Share for TSLA

tsla3_df['Avg Price per Share']=pd.DataFrame(avg_price(tsla3_df))

tsla3_df

,Date,Ticker,Transaction,Transaction Price,Number of shares,Transaction Cost,Close,Net Shares,Cumulative Shares,Avg Price per Share
0,2013-09-23,TSLA,B,189.9,100.0,18990,181.11,100.0,100.0,189.900000
1,2013-09-24,TSLA,NaN,NaN,0.0,NaN,182.33,-0.0,100.0,189.900000
2,2013-09-25,TSLA,NaN,NaN,0.0,NaN,185.24,-0.0,100.0,189.900000
3,2013-09-26,TSLA,NaN,NaN,0.0,NaN,188.64,-0.0,100.0,189.900000
4,2013-09-27,TSLA,NaN,NaN,0.0,NaN,190.90,-0.0,100.0,189.900000
5,2013-09-30,TSLA,NaN,NaN,0.0,NaN,193.37,-0.0,100.0,189.900000
6,2013-10-01,TSLA,NaN,NaN,0.0,NaN,193.00,-0.0,100.0,189.900000
7,2013-10-02,TSLA,NaN,NaN,0.0,NaN,180.95,-0.0,100.0,189.900000
8,2013-10-03,TSLA,NaN,NaN,0.0,NaN,173.31,-0.0,100.0,189.900000
9,2013-10-04,TSLA,NaN,NaN,0.0,NaN,180.98,-0.0,100.0,189.900000


In [107]:
# Create DF for GOOG

goog3_df = portfolio3_df[portfolio3_df['Ticker']=='GOOG']
goog3_df=goog3_df.reset_index(drop=True)

# Run function to create Avg Cost per Share for GOOG

goog3_df['Avg Price per Share']=pd.DataFrame(avg_price(goog3_df))

goog3_df

,Date,Ticker,Transaction,Transaction Price,Number of shares,Transaction Cost,Close,Net Shares,Cumulative Shares,Avg Price per Share
0,2017-08-07,GOOG,B,915.39,10.0,9153.9,929.36,10.0,10.0,915.39
1,2017-08-08,GOOG,NaN,NaN,0.0,NaN,926.79,-0.0,10.0,915.39
2,2017-08-09,GOOG,NaN,NaN,0.0,NaN,922.90,-0.0,10.0,915.39
3,2017-08-10,GOOG,NaN,NaN,0.0,NaN,907.24,-0.0,10.0,915.39
4,2017-08-11,GOOG,NaN,NaN,0.0,NaN,914.39,-0.0,10.0,915.39
5,2017-08-14,GOOG,NaN,NaN,0.0,NaN,922.67,-0.0,10.0,915.39
6,2017-08-15,GOOG,NaN,NaN,0.0,NaN,922.22,-0.0,10.0,915.39
7,2017-08-16,GOOG,NaN,NaN,0.0,NaN,926.96,-0.0,10.0,915.39
8,2017-08-17,GOOG,NaN,NaN,0.0,NaN,910.98,-0.0,10.0,915.39
9,2017-08-18,GOOG,NaN,NaN,0.0,NaN,910.67,-0.0,10.0,915.39


In [111]:
# Merge all 3 tickers to new dataframe

portfolio4_df=pd.concat([aapl3_df, goog3_df, tsla3_df],ignore_index=True)
portfolio4_df=portfolio4_df.sort_values('Date')
portfolio4_df=portfolio4_df.reset_index(drop=True)

portfolio4_df

,Date,Ticker,Transaction,Transaction Price,Number of shares,Transaction Cost,Close,Net Shares,Cumulative Shares,Avg Price per Share
0,2012-03-05,AAPL,B,77.8,100.0,7780,76.17,100.0,100.0,77.800000
1,2012-03-06,AAPL,NaN,NaN,0.0,NaN,75.75,-0.0,100.0,77.800000
2,2012-03-07,AAPL,NaN,NaN,0.0,NaN,75.81,-0.0,100.0,77.800000
3,2012-03-08,AAPL,NaN,NaN,0.0,NaN,77.43,-0.0,100.0,77.800000
4,2012-03-09,AAPL,NaN,NaN,0.0,NaN,77.88,-0.0,100.0,77.800000
5,2012-03-12,AAPL,NaN,NaN,0.0,NaN,78.86,-0.0,100.0,77.800000
6,2012-03-13,AAPL,NaN,NaN,0.0,NaN,81.16,-0.0,100.0,77.800000
7,2012-03-14,AAPL,NaN,NaN,0.0,NaN,84.23,-0.0,100.0,77.800000
8,2012-03-15,AAPL,NaN,NaN,0.0,NaN,83.65,-0.0,100.0,77.800000
9,2012-03-16,AAPL,NaN,NaN,0.0,NaN,83.65,-0.0,100.0,77.800000


In [112]:
# Calculate Cumulative Contribution Column based on Cumulative Shares and Avg Price per Share

portfolio4_df["Cumulative Contributions"] = portfolio4_df["Cumulative Shares"] * portfolio4_df["Avg Price per Share"]

# Calculate Daily Balance Column based on Cumulative Shares and Daily Closing Price

portfolio4_df["Daily Balance"] = portfolio4_df["Cumulative Shares"] * portfolio4_df["Close"]

# Calculate Unrealized Gains (Losses) based on Daily Balance and Cumulative Contributions

portfolio4_df["Unrealized Gains(Losses)"] = portfolio4_df["Daily Balance"] - portfolio4_df["Cumulative Contributions"]

portfolio4_df

,Date,Ticker,Transaction,Transaction Price,Number of shares,Transaction Cost,Close,Net Shares,Cumulative Shares,Avg Price per Share,Cumulative Contributions,Daily Balance,Unrealized Gains(Losses)
0,2012-03-05,AAPL,B,77.8,100.0,7780,76.17,100.0,100.0,77.800000,7780.00,7617.00,-163.00
1,2012-03-06,AAPL,NaN,NaN,0.0,NaN,75.75,-0.0,100.0,77.800000,7780.00,7575.00,-205.00
2,2012-03-07,AAPL,NaN,NaN,0.0,NaN,75.81,-0.0,100.0,77.800000,7780.00,7581.00,-199.00
3,2012-03-08,AAPL,NaN,NaN,0.0,NaN,77.43,-0.0,100.0,77.800000,7780.00,7743.00,-37.00
4,2012-03-09,AAPL,NaN,NaN,0.0,NaN,77.88,-0.0,100.0,77.800000,7780.00,7788.00,8.00
5,2012-03-12,AAPL,NaN,NaN,0.0,NaN,78.86,-0.0,100.0,77.800000,7780.00,7886.00,106.00
6,2012-03-13,AAPL,NaN,NaN,0.0,NaN,81.16,-0.0,100.0,77.800000,7780.00,8116.00,336.00
7,2012-03-14,AAPL,NaN,NaN,0.0,NaN,84.23,-0.0,100.0,77.800000,7780.00,8423.00,643.00
8,2012-03-15,AAPL,NaN,NaN,0.0,NaN,83.65,-0.0,100.0,77.800000,7780.00,8365.00,585.00
9,2012-03-16,AAPL,NaN,NaN,0.0,NaN,83.65,-0.0,100.0,77.800000,7780.00,8365.00,585.00


In [137]:
portfolio_df_returns = portfolio4_df.drop(columns=['Transaction','Transaction Price', 'Number of shares', 'Transaction Cost', 'Close', 'Net Shares', 'Cumulative Shares'])
portfolio_df_returns.drop(columns=['Avg Price per Share','Cumulative Contributions', 'Unrealized Gains(Losses)'], inplace=True)

In [138]:
portfolio_df_returns['Returns'] = pd.DataFrame(portfolio_df_returns['Daily Balance'].pct_change())
portfolio_df_returns

,Date,Ticker,Daily Balance,Returns
0,2012-03-05,AAPL,7617.00,NaN
1,2012-03-06,AAPL,7575.00,-0.005514
2,2012-03-07,AAPL,7581.00,0.000792
3,2012-03-08,AAPL,7743.00,0.021369
4,2012-03-09,AAPL,7788.00,0.005812
5,2012-03-12,AAPL,7886.00,0.012583
6,2012-03-13,AAPL,8116.00,0.029166
7,2012-03-14,AAPL,8423.00,0.037827
8,2012-03-15,AAPL,8365.00,-0.006886
9,2012-03-16,AAPL,8365.00,0.000000


In [139]:
portfolio_df_returns.drop(columns=['Daily Balance'], inplace=True)

In [140]:
portfolio_df_returns.sort_values('Ticker')

,Date,Ticker,Returns
0,2012-03-05,AAPL,NaN
2093,2017-02-08,AAPL,0.832057
2095,2017-02-09,AAPL,0.788734
2097,2017-02-10,AAPL,0.784483
2099,2017-02-13,AAPL,0.727338
2100,2017-02-14,AAPL,0.012979
2103,2017-02-15,AAPL,0.761380
2105,2017-02-16,AAPL,0.830012
2107,2017-02-17,AAPL,0.812906
2109,2017-02-21,AAPL,0.792029


In [143]:
def returns_plot(df):
    return df.hvplot.line(
        x="Date",
        y="Daily Balance",
        groupby= "Ticker").opts(title="Returns")

returns_plot(portfolio4_df)

:DynamicMap   [Ticker]
   :Curve   [Date]   (Daily Balance)

In [145]:
def pct_change_plot(df):
    return df.hvplot(
        x="Date",
        y="Returns",
        groupby= "Ticker").opts(title="Returns Daily Percent Change")

pct_change_plot(portfolio_df_returns)

:DynamicMap   [Ticker]
   :Curve   [Date]   (Returns)

# Insert Heena's Code

In [158]:
# Define last closing price date

max_date=portfolio4_df["Date"].max()

max_date

Timestamp('2020-06-16 00:00:00')

In [159]:
# Create new dataframe with Current Values for Total Shares, Daily Balance, and Cumm Contribution

holdings_df=portfolio4_df[portfolio4_df.Date == max_date]
holdings_df=holdings_df.drop(columns=["Date", "Transaction", "Transaction Price", "Number of shares", "Transaction Cost", "Close", "Net Shares", "Avg Price per Share", "Unrealized Gains(Losses)"]).sort_values("Ticker")

holdings_df

,Ticker,Cumulative Shares,Cumulative Contributions,Daily Balance
4497,AAPL,150.0,10377.75,52812.00
4498,GOOG,10.0,9153.90,14427.20
4499,TSLA,55.0,25230.00,54017.15


In [160]:
# Create new dataframe with Industry & Sector information by Ticker

sector_df=ticker_stats_df_T[["Industry", "Sector"]].reset_index()
sector_df.rename(columns={"index":"Ticker"}, inplace=True)

sector_df

,Ticker,Industry,Sector
0,AAPL,Consumer Electronics,Technology
1,GOOG,Internet Content & Information,Communication Services
2,TSLA,Auto Manufacturers,Consumer Cyclical


In [161]:
# Create new dataframe with Current Values and Industry/Sector information by Ticker

current_industry_holdings = pd.merge(holdings_df, sector_df, on="Ticker")

current_industry_holdings

,Ticker,Cumulative Shares,Cumulative Contributions,Daily Balance,Industry,Sector
0,AAPL,150.0,10377.75,52812.00,Consumer Electronics,Technology
1,GOOG,10.0,9153.90,14427.20,Internet Content & Information,Communication Services
2,TSLA,55.0,25230.00,54017.15,Auto Manufacturers,Consumer Cyclical


In [221]:
# Create Pie Charts for Current Value, Contributions, and Total Shares by Ticker

holdings_contribution_pie=px.pie(current_industry_holdings, values="Cumulative Contributions", names="Ticker", title="Contributions by Stock")
# holdings_shares_pie=px.pie(current_industry_holdings, values="Cumulative Shares", names="Ticker", title="Shares Owned by Stock").update_layout(showlegend=False)
holdings_value_pie=px.pie(current_industry_holdings, values="Daily Balance", names="Ticker", title="Current Value by Stock").update_layout(showlegend=False)


In [222]:
# Create new dataframe for Industry holdings

industry_holdings = current_industry_holdings.groupby("Industry").sum()
industry_holdings.reset_index(inplace=True)

industry_holdings

,Industry,Cumulative Shares,Cumulative Contributions,Daily Balance
0,Auto Manufacturers,55.0,25230.00,54017.15
1,Consumer Electronics,150.0,10377.75,52812.00
2,Internet Content & Information,10.0,9153.90,14427.20


In [223]:
# Create Pie Charts for Current Value, Contributions, and Total Shares by Industry

industry_contribution_pie=px.pie(industry_holdings, values="Cumulative Contributions", names="Industry", title="Contributions by Industry")
# industry_shares_pie=px.pie(industry_holdings, values="Cumulative Shares", names="Industry", title="Shares Owned by Industry").update_layout(showlegend=False)
industry_value_pie=px.pie(industry_holdings, values="Daily Balance", names="Industry", title="Current Value by Industry").update_layout(showlegend=False)


In [224]:
# Create new dataframe for Sector holdings

sector_holdings= current_industry_holdings.groupby("Sector").sum()
sector_holdings.reset_index(inplace=True)

sector_holdings

,Sector,Cumulative Shares,Cumulative Contributions,Daily Balance
0,Communication Services,10.0,9153.90,14427.20
1,Consumer Cyclical,55.0,25230.00,54017.15
2,Technology,150.0,10377.75,52812.00


In [225]:
# Create Pie Charts for Current Value, Contributions, and Total Shares by Sector

sector_contribution_pie=px.pie(sector_holdings, values="Cumulative Contributions", names="Sector", title="Contributions by Sector")
# sector_shares_pie=px.pie(sector_holdings, values="Cumulative Shares", names="Sector", title="Shares Owned by Sector").update_layout(showlegend=False)
sector_value_pie=px.pie(sector_holdings, values="Daily Balance", names="Sector", title="Current Value by Sector").update_layout(showlegend=False)


## Panel Dashboard

Combine all of the pie charts into a single dashboard view using Panel.

In [229]:
# Assign Content to dashboard

industry_analysis=pn.Row(industry_contribution_pie, industry_value_pie, industry_shares_pie)
sector_analysis=pn.Row(sector_contribution_pie, sector_value_pie, sector_shares_pie)
combo_analysis=pn.Column(industry_analysis, sector_analysis)
current_holdings=pn.Row(holdings_contribution_pie, holdings_value_pie, holdings_shares_pie)


In [230]:
# Create the Tabs

dashboard=pn.Tabs(("Ticker Performance", returns_plot(portfolio4_df)),
                  ("Current Holdings", current_holdings),
                  ("Industry & Sector Analysis", combo_analysis))


In [231]:
# Serve the Panel Dashboard

dashboard.servable()

Tabs
    [0] Row(name='Ticker Performance')
        [0] HoloViews(DynamicMap, name='Ticker Performance')
        [1] Column
            [0] WidgetBox
                [0] Select(margin=(20, 20, 20, 20), name='Ticker', options=['AAPL', 'TSLA', 'GOOG'], value='AAPL', width=250)
            [1] VSpacer()
    [1] Row
        [0] Plotly(Figure)
        [1] Plotly(Figure)
        [2] Plotly(Figure)
    [2] Column
        [0] Row
            [0] Plotly(Figure)
            [1] Plotly(Figure)
            [2] Plotly(Figure)
        [1] Row
            [0] Plotly(Figure)
            [1] Plotly(Figure)
            [2] Plotly(Figure)

# Interactive Visualization

Combine all of the pie charts into a dropdown view using Panel.

In [ ]:
# # Define function to choose a Chart View

# def choose_pie_chart(Views):
    
#     industry_analysis_group=pn.Column(industry_value(), industry_contribution(), industry_shares())
#     sector_analysis_group=pn.Column(sector_value(), sector_contribution(), sector_shares())
#     current_holdings_group=pn.Column(holdings_value(), holdings_contribution(), holdings_shares())
    
#     def industry_contribution():
#         industry_contribution_pie=px.pie(industry_holdings, values="Cumm Contribution", names="Industry", title="Contributions by Industry")
#         return industry_contribution_pie

#     def industry_value():
#         industry_value_pie=px.pie(industry_holdings, values="Daily Balance", names="Industry", title="Current Value by Industry")
#         return industry_value_pie

#     def industry_shares():
#         industry_shares_pie=px.pie(industry_holdings, values="Total Shares", names="Industry", title="Shares Owned by Industry")
#         return industry_shares_pie

#     def sector_contribution():
#         sector_contribution_pie=px.pie(sector_holdings, values="Cumm Contribution", names="Sector", title="Contributions by Sector")
#         return sector_contribution_pie

#     def sector_value():
#         sector_value_pie=px.pie(sector_holdings, values="Daily Balance", names="Sector", title="Current Value by Sector")
#         return sector_value_pie

#     def sector_shares():
#         sector_shares_pie=px.pie(sector_holdings, values="Total Shares", names="Sector", title="Shares Owned by Sector")
#         return sector_shares_pie

#     def holdings_contribution():
#         holdings_contribution_pie=px.pie(current_industry_holdings, values="Cumm Contribution", names="Ticker", title="Contributions by Stock")
#         return holdings_contribution_pie

#     def holdings_value():
#         holdings_value_pie=px.pie(current_industry_holdings, values="Daily Balance", names="Ticker", title="Current Value by Stock")
#         return holdings_value_pie

#     def holdings_shares():
#         holdings_shares_pie=px.pie(current_industry_holdings, values="Total Shares", names="Ticker", title="Shares Owned by Stock")
#         return holdings_shares_pie

#     return interact(choose_pie_chart, Views=pie_chart_list)

# # Call the choose_pie_chart function to return plots with interactivity

# pie_chart_list=[industry_analysis_group, sector_analysis_group, current_holdings_group]

# interact(choose_pie_chart, Views=pie_chart_list)


In [ ]:
# # Old pie chart configurations

# industry_contribution_pie2=industry_holdings.plot(kind="pie", y="Cumm Contribution", figsize=(10, 10), legend=True, title="Contributions by Industry", autopct='%1.1f%%')
# industry_shares_pie2=industry_holdings.plot(kind="pie", y="Total Shares", figsize=(10, 10), legend=True, title="Shares Owned by Industry", autopct='%1.1f%%')
# industry_value_pie2=industry_holdings.plot(kind="pie", y="Daily Balance", figsize=(10, 10), legend=True, title="Current Value by Industry", autopct='%1.1f%%')

# sector_contribution_pie=sector_holdings.plot(kind="pie", y="Cumm Contribution", figsize=(10, 10), legend=True, title="Contributions by Sector", autopct='%1.1f%%')
# sector_shares_pie=sector_holdings.plot(kind="pie", y="Total Shares", figsize=(10, 10), legend=True, title="Shares Owned by Sector", autopct='%1.1f%%')
# sector_value_pie=sector_holdings.plot(kind="pie", y="Daily Balance", figsize=(10, 10), legend=True, title="Current Value by Sector", autopct='%1.1f%%')

# holdings_contribution_pie=current_holdings_df.plot(kind="pie", labels=current_holdings_df["Ticker"], y="Cumm Contribution", figsize=(10, 10), legend=True, title="Contributions by Stock", autopct='%1.1f%%')
# holdings_shares_pie=current_holdings_df.plot(kind="pie", labels=current_holdings_df["Ticker"], y="Total Shares", figsize=(10, 10), legend=True, title="Shares Owned by Stock", autopct='%1.1f%%')
# holdings_value_pie=current_holdings_df.plot(kind="pie", labels=current_holdings_df["Ticker"], y="Daily Balance", figsize=(10, 10), legend=True, title="Current Value by Stock", autopct='%1.1f%%')

# Create Ticker Stats Panel
Using the portfolio database, determine the number of unique stocks and pull stock history.
* Inputs:
    * Transposed Ticker Stat Database (ticker_stats_df_T) (Tickers as Columns)
* Outputs:
    * Ticker Stat Panel (stats_panel)

In [232]:
# Define function that will change negative numbers to red text
def color_negative_red(value):
    if value < 0:
        color='red'
    else:
        color='black'
    return 'color: %s' % color

# Define function that will highlight the max value
def highlight_max(value):
    is_max = value == value.max()
    return ['background-color: yellow' if v else '' for v in is_max]

In [233]:
#Format ticker_stats_df_T to:
# Highlight negative values red
# Highlight larget value yellow
# Add '$' and ',' to currency
# Change percentages to %
# Change floats to 2 decimal places
formatted_stats_df = ticker_stats_df_T.style.applymap(
    color_negative_red, subset=['Beta', 'Cash', 'PE Ratio', 'Profit Margin']
    ).apply(
    highlight_max, subset=['Beta', 'Cash', 'Last Dividend', 'PE Ratio', 'Profit Margin']
    ).format(
    {'Beta':'{:,.2f}', 'Cash':'${:,}', 'Last Dividend':'${:,.2f}', 'PE Ratio':'{:,.2f}', 'Profit Margin':'{:,.1%}'}
    )
formatted_stats_df

,PE Ratio,Profit Margin,Cash,Beta,Last Dividend,Industry,Sector
AAPL,20.81,21.2%,"$48,844,000,000",1.23,$3.08,Consumer Electronics,Technology
GOOG,14.70,21.2%,"$18,498,000,000",1.02,$0.00,Internet Content & Information,Communication Services
TSLA,-131.58,-3.5%,"$6,268,000,000",0.58,$0.00,Auto Manufacturers,Consumer Cyclical


In [234]:
stats_panel = pn.panel(formatted_stats_df)
stats_panel

HTML(Styler)

In [ ]:
# code below is my scratchpad area, ignore

In [ ]:
for row in pf:
    if row[index] == 'B':
        lll
    else:
        lll
return lll

In [137]:
stock_dict_copy2=pd.DataFrame(stock_dict_copy)
stock_dict_copy2.to_csv('stock_dict_copy.csv')

In [140]:
portfolio3_df_copy=pd.DataFrame(portfolio3_df)
portfolio3_df_copy.to_csv('portfolio3_df.csv')